In [1]:
import numpy as np
import lsst.sims.featureScheduler as fs
from lsst.sims.speedObservatory import Speed_observatory
import healpy as hp
%matplotlib inline

In [2]:
survey_length = 2  # days
# Define what we want the final visit ratio map to look like
survey_filters = ['u']
surveys = []

for f in survey_filters:
    bfs = []
    bfs.append(fs.Slewtime_basis_function_cost(filtername=f))
    bfs.append(fs.Visit_repeat_basis_function_cost(filtername=f,survey_filters=survey_filters))
    bfs.append(fs.Target_map_basis_function_cost(filtername=f, survey_filters=survey_filters))
    bfs.append(fs.Normalized_alt_basis_function_cost(filtername=f))
    bfs.append(fs.Hour_angle_basis_function_cost())
    bfs.append(fs.Depth_percentile_basis_function_cost())
    weights = np.array([[ 5.,          0.07568279,  8.28700042,  0.66221907,  6.82748119,  5.23734688]])
    surveys.append(fs.Simple_greedy_survey_fields_cost(bfs, weights, filtername=f, block_size= 10))

scheduler = fs.Core_scheduler_cost(surveys)
observatory = Speed_observatory()
observatory, scheduler, observations = fs.sim_runner(observatory, scheduler,
                                                     survey_length=survey_length,
                                                     filename='pairs_survey.db', delete_past=True)

/Users/elahesadatnaghib/Dropbox/Graduate/Research/Princeton/LSSTscheduler/LSST-Code/PythonCode/sims_featureScheduler/python/lsst/sims/featureScheduler/features.py:405: RuntimeWarning: invalid value encountered in arccos
  self.ha_limit = np.arccos(cosha)*12/np.pi
/Users/elahesadatnaghib/Dropbox/Graduate/Research/Princeton/LSSTscheduler/LSST-Code/PythonCode/sims_featureScheduler/python/lsst/sims/featureScheduler/features.py:359: RuntimeWarning: invalid value encountered in arccos
  self.lmst_max = np.arccos(cos_ha) + self.ra
/Users/elahesadatnaghib/gitRepos/sims_ocs/python/lsst/sims/ocs/environment/seeing_model.py:61: RuntimeWarning: invalid value encountered in power
  airmass_correction = numpy.power(airmass, self.AIRMASS_CORRECTION_POWER)
/Users/elahesadatnaghib/Dropbox/Graduate/Research/Princeton/LSSTscheduler/LSST-Code/PythonCode/sims_featureScheduler/python/lsst/sims/featureScheduler/features.py:449: RuntimeWarning: invalid value encountered in greater
  self.feature[np.where(self

ValueError: operands could not be broadcast together with shapes (49152,) (6,) (49152,) 

In [ ]:
for fn, survey in zip(['u', 'g'], scheduler.surveys):
    for basis_function in survey.basis_functions:
        reward = basis_function()
        if np.size(reward) > 1:
            hp.mollview(reward, min=-1, title='$%s$, %s' % (fn, type(basis_function).__name__.split('.')[-1]))

In [ ]:
t_decimals = np.modf(observations['mjd'])[0]
t_dec_shifted = np.roll(t_decimals, -1)
deltas = t_dec_shifted - t_decimals
interval = np.sum(deltas[deltas>0]) *24.*60.*60.
# avg of slew times
avg_slew = np.sum(observations['slewtime'])/interval
# avg number of observations
N_obs = len(observations['RA'])/(interval/60.)

print(interval,avg_slew, N_obs)